In [ ]:
import pandas as pd
import pyodbc as pdb
import psycopg2
import numpy as np

### CONEXIONES

In [ ]:
host = "localhost"
database = "root"
user = "root"
password = "root"
port = "5432"

In [ ]:
connection_target = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )

# Cursor
cur_target = connection_target.cursor()

In [ ]:
query_jubilados = """
select 
  jubilado_id, 
  nombre, 
  apellido, 
  edad, 
  pension_anual, 
  años_tributados, 
  numero_propiedades, 
  nacionalidad_española, 
  g.geografico_id, 
  g.ciudad, 
  g.comunidad_autonoma, 
  g.es_costa, 
  cantidad_hijos, 
  ec.estado_civil, 
  participacion_anterior, 
  maltrato, 
  d.tipo_discapacidad, 
  e.enfermedad_nivel, 
  hj.historial_judicial, 
  endeudamiento, 
  participacion_voluntariado, 
  fumador, 
  preferencia_internacional, 
  tt.tipo_turismo 
from 
  jubilados j 
  left join geografico g on j.geografico_id = g.geografico_id 
  left join estado_civil ec on j.estado_civil_id = ec.estado_civil_id 
  left join discapacidad d on j.tipo_discapacidad_id = d.tipo_discapacidad_id 
  left join enfermedades e on j.enfermedad_id = e.enfermedad_id 
  left join historial_judicial hj on j.historial_judicial_id = hj.historial_judicial_id 
  left join tipo_turismo tt on j.tipo_turismo_id = tt.tipo_turismo_id;
"""

cur_target.execute(query_jubilados)
rows = cur_target.fetchall()
list_of_rows = []
# Para guardar todas las filas
for row in rows:
    list_of_rows.append(row)
# Obtener los nombres de las columnas desde la descripción del cursor
column_names = [desc[0] for desc in cur_target.description]
# Crea el DataFrame con los datos y los nombres de las columnas
jubilados_df = pd.DataFrame(list_of_rows, columns=column_names)
display(jubilados_df)

In [ ]:
query_viajes = """
select 
  viajes_id, 
  g.geografico_id, 
  g.ciudad, 
  g.comunidad_autonoma, 
  g.es_costa, 
  v.numero_plazas, 
  v.numero_dias, 
  m.mes, 
  v.transporte_pagado, 
  tr.residencia, 
  tt.tipo_turismo 
from 
  viajes v 
  left join geografico g on v.geografico_id = g.geografico_id 
  left join mes m on v.mes_id = m.mes_id 
  left join tipo_residencia tr on v.tipo_residencia_id = tr.tipo_residencia_id 
  left join tipo_turismo tt on v.tipo_turismo_id = tt.tipo_turismo_id;
"""

cur_target.execute(query_viajes)
rows = cur_target.fetchall()
list_of_rows = []
# Para guardar todas las filas
for row in rows:
    list_of_rows.append(row)
# Obtener los nombres de las columnas desde la descripción del cursor
column_names = [desc[0] for desc in cur_target.description]
# Crea el DataFrame con los datos y los nombres de las columnas
viajes_df = pd.DataFrame(list_of_rows, columns=column_names)
print(viajes_df)

In [ ]:
fila_aleatoria = viajes_df.sample(n=1)

# Obtiene el valor de geografico_id del viaje elegido al azar
geografico_id_aleatorio = fila_aleatoria['geografico_id'].values[0]
info_viaje = fila_aleatoria.to_dict(orient='records')[0] #guarda la info del viaje en un diccionario

# Filtra df basándose en geografico_id, de forma que puedan acceder los no viven en ese sitio
df_filtrado = jubilados_df[jubilados_df['geografico_id'] != geografico_id_aleatorio]

### FILTROS

In [ ]:
# PUNTOS INICIALES
df_filtrado['puntos'] = 0
df_filtrado['puntos'] = df_filtrado['puntos'].astype(float)  # Para insertar decimales

In [ ]:
#PUNTOS POR EDAD
df_filtrado['puntos'] += np.minimum(10, 10 * (np.exp(0.01 * (df_filtrado['edad'] - 58)) ** 1.15 - 1) / (np.exp(0.01 * (90 - 58)) ** 1.15 - 1)+ 0.6)
#np.minimum 10 es para que el máximo sea 10, que de igual si tiene 90 o 2332478 años, los puntos van a ser 10

In [ ]:
#PUNTOS POR ENDEUDAMIENTO
df_filtrado.loc[df_filtrado['endeudamiento'] == True, 'puntos'] -= 7

In [ ]:
#PUNTOS POR AÑOS TRIBUTADOS
df_filtrado['puntos'] += ((df_filtrado['años_tributados'] - 15) * 0.37)

In [ ]:
#PUNTOS POR FUMADOR
df_filtrado.loc[df_filtrado['fumador'] == True, 'puntos'] -=0.5

In [ ]:
#PUNTOS POR NUMERO DE PROPIEDADES
df_filtrado['puntos'] -= (df_filtrado['numero_propiedades']* 1)

In [ ]:
#PUNTOS POR PARTICIPACION VOLUNTARIADO
df_filtrado.loc[df_filtrado['participacion_voluntario'] == True, 'puntos'] += 2

In [ ]:
#CAPACIDAD VIAJE
df_sorted = df_filtrado.sort_values(by='puntos', ascending=False) #para ordenar de más puntos a menos
capacidad_viaje = info_viaje['capacidad_viaje'] #crear una variable con la capacidad del viaje a partir del diccionario anterior

# RESULTADO
abuelos_seleccionados = df_sorted.head(capacidad_viaje) #limitamos por capacidad y mostramos
viaje_id = info_viaje['viaje_id']
nombre_archivo = f'ganadores_viaje{viaje_id}.csv'
print(*info_viaje.items(), sep='\n')
display(abuelos_seleccionados)
abuelos_seleccionados.to_csv(nombre_archivo, index=False)
